In [11]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

import pickle

import neurokit2 as nk

import seaborn as sns

%matplotlib qt
mpl.rcParams['lines.linewidth'] = 0.91
plt.style.use('ggplot')
plt.style.use('seaborn-v0_8-whitegrid')

In [12]:
import matplotlib.ticker as mtick

In [13]:
sns.set_context("talk")
sns.set_palette("Set2")

In [14]:
subjects = ["158", "098", "633", "279", "906", "547", "971", "958", "815"]

In [24]:
bursts_HR = pd.read_pickle("/Volumes/Untitled/rehab/data/bursts_HR_ACC.pkl")
bursts_HR["ACC_response"] = bursts_HR["ACC_response"].apply(lambda x: np.array(x))
print(bursts_HR.shape)
bursts_HR.head()

(835, 14)


,Start,End,AUC,PC,transition,Limbs,SIB,sub_ID,ACC_response,HR_response,HR_response_normalized,HR_baseline,HR_peak,HR_peak_latency
0,2024-02-28 23:02:28.765630007,2024-02-28 23:02:31.105629921,6.965556e+03,0.0,None,"{LW, T}",1,158,None,"[81.3763655605152, 79.31300702523212, 77.34783...","[2.9331696699604493, 0.32322226398649434, -2.1...",79.057476,2.933170,6.0
1,2024-02-28 23:03:30.215630054,2024-02-28 23:03:32.105629921,6.534732e+03,0.0,None,"{LW, T}",1,158,"[21.685460424790747, 28.13815298028861, 14.266...","[69.75336400943675, 75.84627452807841, 75.8488...","[-8.115480410659373, -0.08942799216255537, -0....",75.914163,3.951068,44.0
2,2024-02-28 23:08:15.275629997,2024-02-28 23:08:15.635629892,7.761742e+02,0.0,None,{LW},1,158,"[-20.92211175552717, -1.6028287851219574, 1.47...","[73.69902935823436, 78.12130225835978, 78.1519...","[1.685935815921141, 7.787548852055352, 7.82978...",72.477112,9.990368,28.0
3,2024-02-28 23:17:41.875629902,2024-02-28 23:17:42.445630074,1.330175e+03,0.0,None,{LW},1,158,"[-11.010669724082462, 13.495555903064954, 0.63...","[75.84328019638515, 78.12715965978558, 73.6972...","[1.36255449131464, 4.414899484319747, -1.50556...",74.823766,8.753980,7.0
4,2024-02-28 23:31:54.005630016,2024-02-28 23:32:11.155630112,1.179498e+06,0.0,None,"{RW, T, LL, RL, LW}",1,158,"[43.69190609430005, 14.230339833499826, 28.264...","[73.69810238588468, 72.33059595781911, 78.1514...","[-1.7699620455902618, -3.5926712332491917, 4.1...",75.026035,38.834153,20.0


# Subject-wise plots

In [56]:
# Single limbs
bursts_HR_single_limbs =  bursts_HR[(bursts_HR["Limbs"] == {"LW"}) | (bursts_HR["Limbs"] == {"RW"}) | (bursts_HR["Limbs"] == {"RL"}) | (bursts_HR["Limbs"] == {"LL"}) | (bursts_HR["Limbs"] == {"T"})].reset_index(drop=True)
# Whole SPT
HR_response_mean_within_singleLimbs = bursts_HR_single_limbs.groupby("sub_ID")["HR_response_normalized"].mean()

# t = np.arange(-19, 50)

# for i in range(0, len(subjects)):
#     plt.figure(figsize=(19, 11))
#     plt.plot(t, HR_response_mean_within_singleLimbs[subjects[i]], label = subjects[i])
#     plt.legend()

# All limbs
bursts_HR_all_limbs = bursts_HR[bursts_HR["Limbs"] == set(("LL", "LW", "RL", "RW", "T"))].reset_index(drop=True)
# divide in low, med and high HR response based on the 1/3 and 2/3 percentiles
HR_response_percentiles = bursts_HR_all_limbs.groupby("sub_ID")["AUC"].describe(percentiles=[1/3, 2/3])
AUC_33 = HR_response_percentiles["33.3%"]
AUC_66 = HR_response_percentiles["66.7%"]

bursts_HR_all_limbs["HR_response_category"] = "medium"
for i, sub in enumerate(subjects):
    bursts_HR_all_limbs.loc[(bursts_HR_all_limbs["sub_ID"] == sub) & (bursts_HR_all_limbs["AUC"] < AUC_33.loc[sub]), "HR_response_category"] = "low"
    bursts_HR_all_limbs.loc[(bursts_HR_all_limbs["sub_ID"] == sub) & (bursts_HR_all_limbs["AUC"] > AUC_66.loc[sub]), "HR_response_category"] = "high"

HR_response_category_mean_within = bursts_HR_all_limbs.groupby(["sub_ID", "HR_response_category"])["HR_response_normalized"].mean().unstack()

for i in range(0, len(subjects)):
    plt.figure(figsize=(19, 11))
    plt.plot(t, HR_response_category_mean_within.loc[subjects[i]]["high"], label = "high")
    plt.plot(t, HR_response_category_mean_within.loc[subjects[i]]["medium"], label = "medium")
    plt.plot(t, HR_response_category_mean_within.loc[subjects[i]]["low"], label = "low")
    plt.title(subjects[i])



# EDA

In [17]:
# plot distribution of HR_peaks and latencies
fig, ax = plt.subplots(1, 2, figsize=(16, 11))

sns.histplot(bursts_HR["HR_peak"], bins=50, kde=True, ax=ax[0])
ax[0].set_title("Distribution of HR peaks")
ax[0].set_xlabel("HR peak (%)")
ax[0].set_ylabel("Frequency")

sns.histplot(bursts_HR["HR_peak_latency"], bins=50, kde=True, ax=ax[1])
ax[1].set_title("Distribution of HR latencies")
ax[1].set_xlabel("HR latency (s)")
ax[1].set_ylabel("Frequency")

Text(0, 0.5, 'Frequency')

In [107]:
# same but with SIB as hue

df_melted = pd.melt(bursts_HR, id_vars=["SIB"], value_vars=["HR_peak", "HR_peak_latency"], var_name="variable", value_name="value")

fig, ax = plt.subplots(1, 2, figsize=(16, 11))

sns.histplot(df_melted, x="value", hue="SIB", bins=50, kde=True, ax=ax[0])
ax[0].set_title("Distribution of HR peaks")
ax[0].set_xlabel("HR peak (%)")
ax[0].set_ylabel("Frequency")

sns.histplot(df_melted, x="value", hue="SIB", bins=50, kde=True, ax=ax[1])
ax[1].set_title("Distribution of HR latencies")
ax[1].set_xlabel("HR latency (s)")
ax[1].set_ylabel("Frequency")

fig, ax = plt.subplots(1, 2, figsize=(16, 11))

sns.boxplot(data=bursts_HR, x="SIB", y="HR_peak", ax=ax[0])
ax[0].set_title("HR peak by SIB")
ax[0].set_ylabel("HR peak (%)")
ax[0].set_xlabel("SIB")

sns.boxplot(data=bursts_HR, x="SIB", y="HR_peak_latency", ax=ax[1])
ax[1].set_title("HR latency by SIB")
ax[1].set_ylabel("HR latency (s)")
ax[1].set_xlabel("SIB")

Text(0.5, 0, 'SIB')

In [115]:
# remove rows with negative HR_peaks
to_remove = bursts_HR[bursts_HR["HR_peak"] < 0].index
bursts_HR = bursts_HR.drop(to_remove).reset_index(drop=True)

# remove rows with HR_peak smaller than 10 and HR_peak_latency larger than 24
to_remove = bursts_HR[(bursts_HR["HR_peak"] < 10) & (bursts_HR["HR_peak_latency"] > 24)].index
bursts_HR = bursts_HR.drop(to_remove).reset_index(drop=True)

bursts_HR.shape

(772, 13)

In [96]:
# plot distribution of AUC 
fig, ax = plt.subplots(1, 2, figsize=(16, 11))

sns.histplot(bursts_HR["AUC"], bins=50, kde=True, ax=ax[0])
ax[0].set_title("Distribution of AUC")
ax[0].set_xlabel("AUC")
ax[0].set_ylabel("Frequency")

sns.boxplot(data=bursts_HR, x="SIB", y="AUC", ax=ax[1])
ax[1].set_title("AUC by SIB")
ax[1].set_ylabel("AUC")
ax[1].set_xlabel("SIB")

Text(0.5, 0, 'SIB')

In [64]:
# Remove too high AUC values
to_remove = bursts_HR[bursts_HR["AUC"] > 3e6].index
bursts_HR = bursts_HR.drop(to_remove).reset_index(drop=True)

# All limbs - Sleep vs Wake

In [10]:
bursts_HR_all_limbs = bursts_HR[bursts_HR["Limbs"] == set(("LL", "LW", "RL", "RW", "T"))].reset_index(drop=True) # all limbs together
# bursts_HR_all_limbs = bursts_HR # all limbs
bursts_HR_all_limbs_sleep = bursts_HR_all_limbs[bursts_HR_all_limbs["SIB"] == 1].reset_index(drop=True)
bursts_HR_all_limbs_wake = bursts_HR_all_limbs[bursts_HR_all_limbs["SIB"] == 0].reset_index(drop=True)

HR_response_mean_within_sleep = bursts_HR_all_limbs_sleep.groupby("sub_ID")["HR_response_normalized"].mean()
HR_response_mean_within_wake = bursts_HR_all_limbs_wake.groupby("sub_ID")["HR_response_normalized"].mean()

sem_sleep = HR_response_mean_within_sleep.to_numpy().std() / np.sqrt(HR_response_mean_within_sleep.to_numpy().shape[0])
sem_wake = HR_response_mean_within_wake.to_numpy().std() / np.sqrt(HR_response_mean_within_wake.to_numpy().shape[0])

t = np.arange(-19, 50)

f, (ax) = plt.subplots()
f.set_figheight(9)
f.set_figwidth(15)

plt.plot(t, HR_response_mean_within_sleep.mean(), label = "sleep", color = 'blue')
plt.errorbar(t, HR_response_mean_within_sleep.mean(), yerr = sem_sleep, fmt = '-o', color = 'blue', ecolor='blue', capsize=3)

plt.plot(t, HR_response_mean_within_wake.mean(), label = "wake", color = 'red')
plt.errorbar(t, HR_response_mean_within_wake.mean(), yerr = sem_wake, fmt = '-o', color = 'red', ecolor='red', capsize=3)

ax.axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax.axhline(y=0, color='grey', linestyle='--', linewidth=1.2)
ax.annotate('Movement\nonset', xy=(0, plt.ylim()[1]-6), xytext=(-60, plt.ylim()[1]-66),
             textcoords='offset points', ha='right', va='bottom', fontsize=19,
             bbox=dict(boxstyle='round,pad=0.5', fc='gray', alpha=0.2, edgecolor='black'),
             arrowprops=dict(facecolor='black', shrink=0.05, width=2))

# plt.xticks(ticks = np.arange(-20, 40, 5), labels=np.arange(-20, 40, 5), fontsize=16)
ax.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax.set_xlabel('Time (seconds)', fontsize = 21)
ax.set_ylabel('HR change', fontsize = 21)
ax.set_label('')
ax.xaxis.set_tick_params(labelsize=20)
ax.yaxis.set_tick_params(labelsize=20)

plt.legend(frameon = True, fancybox = True, shadow = True, fontsize = 18, loc = "upper right")

HR_peak_all_limbs_sleep = bursts_HR_all_limbs_sleep.groupby("sub_ID")["HR_peak"].mean()
HR_peak_all_limbs_wake = bursts_HR_all_limbs_wake.groupby("sub_ID")["HR_peak"].mean()

HR_peak_all_limbs_sleep.describe(), HR_peak_all_limbs_wake.describe()

(count     9.000000
 mean     32.908198
 std      10.395209
 min      20.242914
 25%      24.385344
 50%      28.349812
 75%      42.366226
 max      48.449517
 Name: HR_peak, dtype: float64,
 count     9.000000
 mean     52.671251
 std      19.810720
 min      28.438842
 25%      37.689244
 50%      58.363721
 75%      64.151162
 max      87.997540
 Name: HR_peak, dtype: float64)

# ALL limbs

In [6]:
bursts_HR_all_limbs = bursts_HR[bursts_HR["Limbs"] == set(("LL", "LW", "RL", "RW", "T"))].reset_index(drop=True)

HR_response_category_mean_within = bursts_HR_all_limbs.groupby("sub_ID")["HR_response_normalized"].mean()

sem = HR_response_category_mean_within.to_numpy().std() / np.sqrt(HR_response_category_mean_within.to_numpy().shape[0])

t = np.arange(-19, 50)
plt.figure(figsize=(15, 9))
plt.plot(t, HR_response_category_mean_within.mean(), color = 'blue')
plt.errorbar(t, HR_response_category_mean_within.mean(), yerr = sem, fmt = '-o', color = 'blue', ecolor='blue', capsize=3)

HR_peak_all_limbs = bursts_HR_all_limbs.groupby("sub_ID")["HR_peak"].mean()
HR_peak_all_limbs.describe()

count     9.000000
mean     40.440675
std      14.389471
min      25.159911
25%      30.088659
50%      33.492819
75%      49.929787
max      60.705317
Name: HR_peak, dtype: float64

# ALL limbs - HIGH vs MED vs LOW

In [20]:
bursts_HR_all_limbs = bursts_HR#[bursts_HR["Limbs"] == set(("LL", "LW", "RL", "RW", "T"))].reset_index(drop=True)

## Whole SPT

In [8]:
# divide in low, med and high HR response based on the 1/3 and 2/3 percentiles
HR_response_percentiles = bursts_HR_all_limbs.groupby("sub_ID")["AUC"].describe(percentiles=[1/3, 2/3])
AUC_33 = HR_response_percentiles["33.3%"]
AUC_66 = HR_response_percentiles["66.7%"]

bursts_HR_all_limbs["HR_response_category"] = "medium"
for i, sub in enumerate(subjects):
    bursts_HR_all_limbs.loc[(bursts_HR_all_limbs["sub_ID"] == sub) & (bursts_HR_all_limbs["AUC"] < AUC_33.loc[sub]), "HR_response_category"] = "low"
    bursts_HR_all_limbs.loc[(bursts_HR_all_limbs["sub_ID"] == sub) & (bursts_HR_all_limbs["AUC"] > AUC_66.loc[sub]), "HR_response_category"] = "high"

In [9]:
# coherent averaging within subjects for each category
HR_response_category_mean_within = bursts_HR_all_limbs.groupby(["sub_ID", "HR_response_category"])["HR_response_normalized"].mean().unstack()

sem_high = HR_response_category_mean_within["high"].to_numpy().std() / np.sqrt(HR_response_category_mean_within["high"].to_numpy().shape[0])
sem_medium = HR_response_category_mean_within["medium"].to_numpy().std() / np.sqrt(HR_response_category_mean_within["medium"].to_numpy().shape[0])
sem_low = HR_response_category_mean_within["low"].to_numpy().std() / np.sqrt(HR_response_category_mean_within["low"].to_numpy().shape[0])

t = np.arange(-19, 50, 1)
plt.figure(figsize=(15, 9))
plt.plot(t, HR_response_category_mean_within["low"].mean(), label = "low", color = 'blue')
plt.errorbar(t, HR_response_category_mean_within["low"].mean(), yerr = sem_low, fmt = '-o', color = 'blue', ecolor='blue', capsize=3)

plt.plot(t, HR_response_category_mean_within["medium"].mean(), label = "medium", color = 'orange')
plt.errorbar(t, HR_response_category_mean_within["medium"].mean(), yerr = sem_medium, fmt = '-o', color = 'orange', ecolor='orange', capsize=3)

plt.plot(t, HR_response_category_mean_within["high"].mean(), label = "high", color = 'red')
plt.errorbar(t, HR_response_category_mean_within["high"].mean(), yerr = sem_high, fmt = '-o', color = 'red', ecolor='red', capsize=3)

plt.legend()

### Statistics

In [44]:
HR_peak_category_mean_within = bursts_HR_all_limbs.groupby(["sub_ID", "HR_response_category"])["HR_peak"].mean().unstack()
HR_peak_category_mean_within

# Friedman test
from scipy.stats import friedmanchisquare

stat, p = friedmanchisquare(list(HR_peak_category_mean_within["high"].values), list(HR_peak_category_mean_within["medium"].values), list(HR_peak_category_mean_within["low"].values))
print(f"HR peaks during sleep: p = {p}")
print(p < 0.01)

# Wilcoxon test
from scipy.stats import wilcoxon

stat, p = wilcoxon(list(HR_peak_category_mean_within["high"].values), list(HR_peak_category_mean_within["medium"].values))
print(f"High vs Medium: p = {p}")
print(p < 0.01)

stat, p = wilcoxon(list(HR_peak_category_mean_within["high"].values), list(HR_peak_category_mean_within["low"].values))
print(f"High vs Low: p = {p}")
print(p < 0.01)

stat, p = wilcoxon(list(HR_peak_category_mean_within["medium"].values), list(HR_peak_category_mean_within["low"].values))
print(f"Medium vs Low: p = {p}")
print(p < 0.01)

# plot HR peaks
fig, ax = plt.subplots(1, 2, figsize=(16, 11))

sns.boxplot(data=HR_peak_category_mean_within, ax=ax[0], palette=sns.color_palette("Set2", n_colors=3))
ax[0].set_title("HR peak by HR response category")
ax[0].set_ylabel("HR peak (%)")
ax[0].set_xlabel("HR response category")

# HR_latency_category_mean_within = bursts_HR_all_limbs.groupby(["sub_ID", "HR_response_category"])["HR_peak_latency"].mean().unstack()

# sns.boxplot(data=HR_latency_category_mean_within, ax=ax[1], palette=sns.color_palette("Set2", n_colors=3))
# ax[1].set_title("HR latency by HR response category")
# ax[1].set_ylabel("HR latency (s)")
# ax[1].set_xlabel("HR response category")


HR peaks during sleep: p = 0.0003001851487690379
True
High vs Medium: p = 0.01171875
False
High vs Low: p = 0.00390625
True
Medium vs Low: p = 0.00390625
True


Text(0.5, 0, 'HR response category')

## Sleep

In [21]:
bursts_HR_all_limbs_sleep = bursts_HR_all_limbs[bursts_HR_all_limbs["SIB"] == 1].reset_index(drop=True)

In [22]:
# divide in low, med and high HR response based on the 1/3 and 2/3 percentiles
HR_response_percentiles = bursts_HR_all_limbs_sleep.groupby("sub_ID")["AUC"].describe(percentiles=[1/3, 2/3])
AUC_33_sleep = HR_response_percentiles["33.3%"]
AUC_66_sleep = HR_response_percentiles["66.7%"]

bursts_HR_all_limbs_sleep["HR_response_category"] = "medium"
for i, sub in enumerate(subjects):
    bursts_HR_all_limbs_sleep.loc[(bursts_HR_all_limbs_sleep["sub_ID"] == sub) & (bursts_HR_all_limbs_sleep["AUC"] < AUC_33_sleep.loc[sub]), "HR_response_category"] = "low"
    bursts_HR_all_limbs_sleep.loc[(bursts_HR_all_limbs_sleep["sub_ID"] == sub) & (bursts_HR_all_limbs_sleep["AUC"] > AUC_66_sleep.loc[sub]), "HR_response_category"] = "high"

In [23]:
# coherent averaging within subjects for each category
HR_response_category_mean_within_sleep = bursts_HR_all_limbs_sleep.groupby(["sub_ID", "HR_response_category"])["HR_response_normalized"].mean().unstack()

sem_high = HR_response_category_mean_within_sleep["high"].to_numpy().std() / np.sqrt(HR_response_category_mean_within_sleep["high"].to_numpy().shape[0])
sem_medium = HR_response_category_mean_within_sleep["medium"].to_numpy().std() / np.sqrt(HR_response_category_mean_within_sleep["medium"].to_numpy().shape[0])
sem_low = HR_response_category_mean_within_sleep["low"].to_numpy().std() / np.sqrt(HR_response_category_mean_within_sleep["low"].to_numpy().shape[0])

t = np.arange(-19, 50, 1)
plt.figure(figsize=(15, 9))
plt.plot(t, HR_response_category_mean_within_sleep["low"].mean(), label = "low", color = 'blue')
plt.errorbar(t, HR_response_category_mean_within_sleep["low"].mean(), yerr = sem_low, fmt = '-o', color = 'blue', ecolor='blue', capsize=3)

plt.plot(t, HR_response_category_mean_within_sleep["medium"].mean(), label = "medium", color = 'orange')
plt.errorbar(t, HR_response_category_mean_within_sleep["medium"].mean(), yerr = sem_medium, fmt = '-o', color = 'orange', ecolor='orange', capsize=3)

plt.plot(t, HR_response_category_mean_within_sleep["high"].mean(), label = "high", color = 'red')
plt.errorbar(t, HR_response_category_mean_within_sleep["high"].mean(), yerr = sem_high, fmt = '-o', color = 'red', ecolor='red', capsize=3)

plt.legend()

### Statistics

In [50]:
HR_peak_category_mean_within = bursts_HR_all_limbs_sleep.groupby(["sub_ID", "HR_response_category"])["HR_peak"].mean().unstack()
HR_peak_latency_category_mean_within = bursts_HR_all_limbs_sleep.groupby(["sub_ID", "HR_response_category"])["HR_peak_latency"].mean().unstack()

# Friedman test
from scipy.stats import friedmanchisquare

stat, p = friedmanchisquare(list(HR_peak_category_mean_within["high"].values), list(HR_peak_category_mean_within["medium"].values), list(HR_peak_category_mean_within["low"].values))
print(f"HR peaks during sleep: p = {p}")
print(p < 0.01)

# Wilcoxon test
from scipy.stats import wilcoxon

stat, p = wilcoxon(list(HR_peak_category_mean_within["high"].values), list(HR_peak_category_mean_within["medium"].values))
print(f"High vs Medium: p = {p}")
print(p < 0.01)

stat, p = wilcoxon(list(HR_peak_category_mean_within["high"].values), list(HR_peak_category_mean_within["low"].values))
print(f"High vs Low: p = {p}")
print(p < 0.01)

stat, p = wilcoxon(list(HR_peak_category_mean_within["medium"].values), list(HR_peak_category_mean_within["low"].values))
print(f"Medium vs Low: p = {p}")
print(p < 0.01)

# plot HR peaks
fig, ax = plt.subplots(1, 2, figsize=(16, 11))

sns.boxplot(data=HR_peak_category_mean_within, ax=ax[0], palette=sns.color_palette("Set2", n_colors=3))
sns.stripplot(data=HR_peak_category_mean_within, ax=ax[0], palette=sns.color_palette("Set2", n_colors=3), alpha=0.5, size=8,
              jitter=0.1, edgecolor='black', linewidth=0.5)
ax[0].set_title("HR peak by HR response category")
ax[0].set_ylabel("HR peak (%)")
ax[0].set_xlabel("HR response category")

# Friedman test

stat, p = friedmanchisquare(list(HR_peak_latency_category_mean_within["high"].values), list(HR_peak_latency_category_mean_within["medium"].values), list(HR_peak_latency_category_mean_within["low"].values))
print(f"HR peak latencies during sleep: p = {p}")
print(p < 0.01)

# Wilcoxon test

stat, p = wilcoxon(list(HR_peak_latency_category_mean_within["high"].values), list(HR_peak_latency_category_mean_within["medium"].values))
print(f"High vs Medium: p = {p}")
print(p < 0.01)

stat, p = wilcoxon(list(HR_peak_latency_category_mean_within["high"].values), list(HR_peak_latency_category_mean_within["low"].values))
print(f"High vs Low: p = {p}")
print(p < 0.01)

stat, p = wilcoxon(list(HR_peak_latency_category_mean_within["medium"].values), list(HR_peak_latency_category_mean_within["low"].values))
print(f"Medium vs Low: p = {p}")
print(p < 0.01)

# plot HR latencies
sns.boxplot(data=HR_peak_latency_category_mean_within, palette=sns.color_palette("Set2", n_colors=3), ax=ax[1])
sns.stripplot(data=HR_peak_latency_category_mean_within, ax=ax[1], palette=sns.color_palette("Set2", n_colors=3), alpha=0.5, size=8,
              jitter=0.1, edgecolor='black', linewidth=0.5)
ax[1].set_title("HR latency by HR response category")
ax[1].set_ylabel("HR latency (s)")
ax[1].set_xlabel("HR response category")

HR peaks during sleep: p = 0.00309558685236526
True
High vs Medium: p = 0.00390625
True
High vs Low: p = 0.0078125
True
Medium vs Low: p = 0.1640625
False
HR peak latencies during sleep: p = 0.04455142624448988
False
High vs Medium: p = 0.12890625
False
High vs Low: p = 0.1640625
False
Medium vs Low: p = 0.8203125
False


Text(0.5, 0, 'HR response category')

## Wake

In [13]:
bursts_HR_all_limbs_wake = bursts_HR_all_limbs[bursts_HR_all_limbs["SIB"] == 0].reset_index(drop=True)

In [14]:
# divide in low, med and high HR response based on the 1/3 and 2/3 percentiles
HR_response_percentiles = bursts_HR_all_limbs_wake.groupby("sub_ID")["AUC"].describe(percentiles=[1/3, 2/3])
AUC_33_wake = HR_response_percentiles["33.3%"]
AUC_66_wake = HR_response_percentiles["66.7%"]

bursts_HR_all_limbs_wake["HR_response_category"] = "medium"
for i, sub in enumerate(subjects):
    bursts_HR_all_limbs_wake.loc[(bursts_HR_all_limbs_wake["sub_ID"] == sub) & (bursts_HR_all_limbs_wake["AUC"] < AUC_33_wake.loc[sub]), "HR_response_category"] = "low"
    bursts_HR_all_limbs_wake.loc[(bursts_HR_all_limbs_wake["sub_ID"] == sub) & (bursts_HR_all_limbs_wake["AUC"] > AUC_66_wake.loc[sub]), "HR_response_category"] = "high"

In [15]:
# coherent averaging within subjects for each category
HR_response_category_mean_within_wake = bursts_HR_all_limbs_wake.groupby(["sub_ID", "HR_response_category"])["HR_response_normalized"].mean().unstack()

sem_high = HR_response_category_mean_within_wake["high"].dropna().to_numpy().std() / np.sqrt(HR_response_category_mean_within_wake["high"].dropna().to_numpy().shape[0])
sem_medium = HR_response_category_mean_within_wake["medium"].dropna().to_numpy().std() / np.sqrt(HR_response_category_mean_within_wake["medium"].dropna().to_numpy().shape[0])
sem_low = HR_response_category_mean_within_wake["low"].dropna().to_numpy().std() / np.sqrt(HR_response_category_mean_within_wake["low"].dropna().to_numpy().shape[0])

t = np.arange(-19, 50, 1)
plt.figure(figsize=(15, 9))
plt.plot(t, HR_response_category_mean_within_wake["low"].dropna().mean(), label = "low", color = 'blue')
plt.errorbar(t, HR_response_category_mean_within_wake["low"].dropna().mean(), yerr = sem_low, fmt = '-o', color = 'blue', ecolor='blue', capsize=3)

plt.plot(t, HR_response_category_mean_within_wake["medium"].dropna().mean(), label = "medium", color = 'orange')
plt.errorbar(t, HR_response_category_mean_within_wake["medium"].dropna().mean(), yerr = sem_medium, fmt = '-o', color = 'orange', ecolor='orange', capsize=3)

plt.plot(t, HR_response_category_mean_within_wake["high"].dropna().mean(), label = "high", color = 'red')
plt.errorbar(t, HR_response_category_mean_within_wake["high"].dropna().mean(), yerr = sem_high, fmt = '-o', color = 'red', ecolor='red', capsize=3)

plt.legend()

# Posture Changes

In [75]:
bursts_HR_all_limbs_PC = bursts_HR[bursts_HR["PC"] > 0].reset_index(drop=True)

## All

In [76]:
HR_response_mean_within_PC = bursts_HR_all_limbs_PC.groupby(["sub_ID"])["HR_response_normalized"].mean()

sem = HR_response_mean_within_PC.to_numpy().std() / np.sqrt(HR_response_mean_within_PC.to_numpy().shape[0])

t = np.arange(-19, 50, 1)
plt.figure(figsize=(15, 9))
plt.plot(t, HR_response_mean_within_PC.mean(), label = "PC", color = 'blue')
plt.errorbar(t, HR_response_mean_within_PC.mean(), yerr = sem, fmt = '-o', color = 'blue', ecolor='blue', capsize=3)

<ErrorbarContainer object of 3 artists>

## Stratify by transition type

In [77]:
HR_response_mean_within_PC = bursts_HR_all_limbs_PC.groupby(["sub_ID", "transition"])["HR_response_normalized"].mean()

# average together responses to the same transition
HR_response_mean_within_PC = HR_response_mean_within_PC.groupby("transition").mean()
HR_response_mean_within_PC

# plot each one
trans = bursts_HR_all_limbs_PC["transition"].dropna().unique()
t = np.arange(-19, 50, 1)
plt.figure(figsize=(15, 9)) 
for i, transition in enumerate(trans):
    plt.plot(t, HR_response_mean_within_PC.loc[transition], label = transition, color = sns.color_palette("Set2", n_colors=12)[i])
plt.legend()

# Single Limbs (Either `LW`, `RW`, `LL`, `RL`)

In [16]:
bursts_HR_single_limbs =  bursts_HR[(bursts_HR["Limbs"] == {"LW"}) | (bursts_HR["Limbs"] == {"RW"}) | (bursts_HR["Limbs"] == {"RL"}) | (bursts_HR["Limbs"] == {"LL"}) | (bursts_HR["Limbs"] == {"T"})].reset_index(drop=True)
bursts_HR_single_limbs.groupby("sub_ID")["AUC"].describe()

to_remove = bursts_HR_single_limbs[bursts_HR_single_limbs["sub_ID"] == "633"].index
bursts_HR_single_limbs = bursts_HR_single_limbs.drop(to_remove).reset_index(drop=True)

# discard subjects with AUC < 14
# to_remove = bursts_HR_single_limbs[bursts_HR_single_limbs["AUC"] < 14].index
# bursts_HR_single_limbs = bursts_HR_single_limbs.drop(to_remove).reset_index(drop=True)

In [18]:
# Whole SPT
HR_response_mean_within_singleLimbs = bursts_HR_single_limbs.groupby("sub_ID")["HR_response_normalized"].mean()

sem = HR_response_mean_within_singleLimbs.to_numpy().std() / np.sqrt(HR_response_mean_within_singleLimbs.to_numpy().shape[0])

plt.figure(figsize=(15, 9))
plt.plot(t, HR_response_mean_within_singleLimbs.mean(), label = "single limb", color = 'blue')
plt.errorbar(t, HR_response_mean_within_singleLimbs.mean(), yerr = sem, fmt = '-o', color = 'blue', ecolor='blue', capsize=3)
plt.title("Whole SPT")

# Sleep
HR_response_mean_within_singleLimbs_sleep = bursts_HR_single_limbs[bursts_HR_single_limbs["SIB"] == 1].groupby("sub_ID")["HR_response_normalized"].mean()

sem = HR_response_mean_within_singleLimbs_sleep.to_numpy().std() / np.sqrt(HR_response_mean_within_singleLimbs_sleep.to_numpy().shape[0])

plt.figure(figsize=(15, 9))
plt.plot(t, HR_response_mean_within_singleLimbs_sleep.mean(), label = "single limb", color = 'blue')
plt.errorbar(t, HR_response_mean_within_singleLimbs_sleep.mean(), yerr = sem, fmt = '-o', color = 'blue', ecolor='blue', capsize=3)
plt.title("Sleep")

# Wake
HR_response_mean_within_singleLimbs_wake = bursts_HR_single_limbs[bursts_HR_single_limbs["SIB"] == 0].groupby("sub_ID")["HR_response_normalized"].mean()

sem = HR_response_mean_within_singleLimbs_wake.to_numpy().std() / np.sqrt(HR_response_mean_within_singleLimbs_wake.to_numpy().shape[0])

plt.figure(figsize=(15, 9))
plt.plot(t, HR_response_mean_within_singleLimbs_wake.mean(), label = "single limb", color = 'blue')
plt.errorbar(t, HR_response_mean_within_singleLimbs_wake.mean(), yerr = sem, fmt = '-o', color = 'blue', ecolor='blue', capsize=3)
plt.title("Wake")

Text(0.5, 1.0, 'Wake')

# Single limbs alone (Only `LW`, Only `RW`, Only `LL`, Only `RL`)

In [19]:
bursts_LW = bursts_HR[bursts_HR["Limbs"] == {"LW"}].reset_index(drop=True)
bursts_RW = bursts_HR[bursts_HR["Limbs"] == {"RW"}].reset_index(drop=True)
bursts_LL = bursts_HR[bursts_HR["Limbs"] == {"LL"}].reset_index(drop=True)
bursts_RL = bursts_HR[bursts_HR["Limbs"] == {"RL"}].reset_index(drop=True)
bursts_T = bursts_HR[bursts_HR["Limbs"] == {"T"}].reset_index(drop=True)

HR_response_mean_within_LW = bursts_LW.groupby("sub_ID")["HR_response_normalized"].mean()
HR_response_mean_within_RW = bursts_RW.groupby("sub_ID")["HR_response_normalized"].mean()
HR_response_mean_within_LL = bursts_LL.groupby("sub_ID")["HR_response_normalized"].mean()
HR_response_mean_within_RL = bursts_RL.groupby("sub_ID")["HR_response_normalized"].mean()
HR_response_mean_within_T = bursts_T.groupby("sub_ID")["HR_response_normalized"].mean()

sem_LW = HR_response_mean_within_LW.to_numpy().std() / np.sqrt(HR_response_mean_within_LW.to_numpy().shape[0])
sem_RW = HR_response_mean_within_RW.to_numpy().std() / np.sqrt(HR_response_mean_within_RW.to_numpy().shape[0])
sem_LL = HR_response_mean_within_LL.dropna().to_numpy().std() / np.sqrt(HR_response_mean_within_LL.dropna().to_numpy().shape[0])
sem_RL = HR_response_mean_within_RL.to_numpy().std() / np.sqrt(HR_response_mean_within_RL.to_numpy().shape[0])
sem_T = HR_response_mean_within_T.to_numpy().std() / np.sqrt(HR_response_mean_within_T.to_numpy().shape[0])

plt.figure(figsize=(15, 9))
plt.subplot(2, 2, 1)
plt.plot(t, HR_response_mean_within_LW.mean(), label = "LW", color = 'blue')
plt.errorbar(t, HR_response_mean_within_LW.mean(), yerr = sem_LW, fmt = '-o', color = 'blue', ecolor='blue', capsize=3)
plt.xlim(-12, 28)
plt.title("LW")

plt.subplot(2, 2, 2)
plt.plot(t, HR_response_mean_within_RW.mean(), label = "RW", color = 'orange')
plt.errorbar(t, HR_response_mean_within_RW.mean(), yerr = sem_RW, fmt = '-o', color = 'orange', ecolor='orange', capsize=3)
plt.xlim(-12, 28)
plt.title("RW")

plt.subplot(2, 2, 3)
plt.plot(t, HR_response_mean_within_LL.mean(), label = "LL", color = 'green')
plt.errorbar(t, HR_response_mean_within_LL.mean(), yerr = sem_LL, fmt = '-o', color = 'green', ecolor='green', capsize=3)
plt.xlim(-12, 28)
plt.title("LL")

plt.subplot(2, 2, 4)
plt.plot(t, HR_response_mean_within_RL.mean(), label = "RL", color = 'red')
plt.errorbar(t, HR_response_mean_within_RL.mean(), yerr = sem_RL, fmt = '-o', color = 'red', ecolor='red', capsize=3)
plt.xlim(-12, 28)
plt.title("RL")

plt.figure(figsize=(15, 9))
plt.plot(t, HR_response_mean_within_T.mean(), label = "T", color = 'blue')
plt.errorbar(t, HR_response_mean_within_T.mean(), yerr = sem_T, fmt = '-o', color = 'blue', ecolor='blue', capsize=3)
plt.xlim(-12, 28)
plt.title("T")


plt.tight_layout()

# Lower Body Movements

In [20]:
# With trunk
bursts_lower_body = bursts_HR[(bursts_HR["Limbs"] == {"LL", "RL"}) | (bursts_HR["Limbs"] == {"LL", "RL", "T"})].reset_index(drop=True)

HR_response_mean_within_lower_body = bursts_lower_body.groupby("sub_ID")["HR_response_normalized"].mean()

sem = HR_response_mean_within_lower_body.to_numpy().std() / np.sqrt(HR_response_mean_within_lower_body.to_numpy().shape[0])

plt.figure(figsize=(15, 9))
plt.plot(t, HR_response_mean_within_lower_body.mean(), label = "lower body", color = 'blue')
plt.errorbar(t, HR_response_mean_within_lower_body.mean(), yerr = sem, fmt = '-o', color = 'blue', ecolor='blue', capsize=3)

# No trunk
bursts_lower_body = bursts_HR[(bursts_HR["Limbs"] == {"LL", "RL"})].reset_index(drop=True)

HR_response_mean_within_lower_body = bursts_lower_body.groupby("sub_ID")["HR_response_normalized"].mean()

sem = HR_response_mean_within_lower_body.to_numpy().std() / np.sqrt(HR_response_mean_within_lower_body.to_numpy().shape[0])

plt.figure(figsize=(15, 9))
plt.plot(t, HR_response_mean_within_lower_body.mean(), label = "lower body", color = 'blue')
plt.errorbar(t, HR_response_mean_within_lower_body.mean(), yerr = sem, fmt = '-o', color = 'blue', ecolor='blue', capsize=3)

<ErrorbarContainer object of 3 artists>

# Upper Body Movements

In [21]:
# With trunk
bursts_upper_body = bursts_HR[(bursts_HR["Limbs"] == {"LW", "RW"}) | (bursts_HR["Limbs"] == {"LW", "RW", "T"})].reset_index(drop=True)

HR_response_mean_within_upper_body = bursts_upper_body.groupby("sub_ID")["HR_response_normalized"].mean()

sem = HR_response_mean_within_upper_body.to_numpy().std() / np.sqrt(HR_response_mean_within_upper_body.to_numpy().shape[0])

plt.figure(figsize=(15, 9))
plt.plot(t, HR_response_mean_within_upper_body.mean(), label = "upper body", color = 'blue')
plt.errorbar(t, HR_response_mean_within_upper_body.mean(), yerr = sem, fmt = '-o', color = 'blue', ecolor='blue', capsize=3)

# No trunk
bursts_upper_body = bursts_HR[(bursts_HR["Limbs"] == {"LW", "RW"})].reset_index(drop=True)

HR_response_mean_within_upper_body = bursts_upper_body.groupby("sub_ID")["HR_response_normalized"].mean()

sem = HR_response_mean_within_upper_body.to_numpy().std() / np.sqrt(HR_response_mean_within_upper_body.to_numpy().shape[0])

plt.figure(figsize=(15, 9))
plt.plot(t, HR_response_mean_within_upper_body.mean(), label = "upper body", color = 'blue')
plt.errorbar(t, HR_response_mean_within_upper_body.mean(), yerr = sem, fmt = '-o', color = 'blue', ecolor='blue', capsize=3)

<ErrorbarContainer object of 3 artists>

# Unilateral movements

In [24]:
bursts_left = bursts_HR[(bursts_HR["Limbs"] == {"LL", "LW"}) | (bursts_HR["Limbs"] == {"LL", "LW", "T"})].reset_index(drop=True)

HR_response_mean_within_left = bursts_left.groupby("sub_ID")["HR_response_normalized"].mean()

sem = HR_response_mean_within_left.dropna().to_numpy().std() / np.sqrt(HR_response_mean_within_left.dropna().to_numpy().shape[0])

plt.figure(figsize=(15, 9))
plt.plot(t, HR_response_mean_within_left.mean(), label = "left", color = 'blue')
plt.errorbar(t, HR_response_mean_within_left.mean(), yerr = sem, fmt = '-o', color = 'blue', ecolor='blue', capsize=3)

bursts_right = bursts_HR[(bursts_HR["Limbs"] == {"RL", "RW"}) | (bursts_HR["Limbs"] == {"RL", "RW", "T"})].reset_index(drop=True)

HR_response_mean_within_right = bursts_right.groupby("sub_ID")["HR_response_normalized"].mean()

sem = HR_response_mean_within_right.to_numpy().std() / np.sqrt(HR_response_mean_within_right.to_numpy().shape[0])

plt.figure(figsize=(15, 9))
plt.plot(t, HR_response_mean_within_right.mean(), label = "right", color = 'blue')
plt.errorbar(t, HR_response_mean_within_right.mean(), yerr = sem, fmt = '-o', color = 'blue', ecolor='blue', capsize=3)

<ErrorbarContainer object of 3 artists>

# All movements except 5 limbs

In [26]:
bursts_HR_all_limbs = bursts_HR[bursts_HR["Limbs"] != set(("LL", "LW", "RL", "RW", "T"))].reset_index(drop=True)

HR_response_category_mean_within = bursts_HR_all_limbs.groupby("sub_ID")["HR_response_normalized"].mean()

sem = HR_response_category_mean_within.to_numpy().std() / np.sqrt(HR_response_category_mean_within.to_numpy().shape[0])

t = np.arange(-19, 50)
plt.figure(figsize=(15, 9))
plt.plot(t, HR_response_category_mean_within.mean(), color = 'blue')
plt.errorbar(t, HR_response_category_mean_within.mean(), yerr = sem, fmt = '-o', color = 'blue', ecolor='blue', capsize=3)

HR_peak_all_limbs = bursts_HR_all_limbs.groupby("sub_ID")["HR_peak"].mean()
HR_peak_all_limbs.describe()

count     9.000000
mean     18.941544
std       8.765316
min      10.831330
25%      13.632555
50%      18.461241
75%      19.442349
max      40.367206
Name: HR_peak, dtype: float64

# Acc 

## Subject-wise plots

In [95]:
# Single limbs
bursts_ACC_single_limbs =  bursts_HR[(bursts_HR["Limbs"] == {"LW"}) | (bursts_HR["Limbs"] == {"RW"}) | (bursts_HR["Limbs"] == {"RL"}) | (bursts_HR["Limbs"] == {"LL"}) | (bursts_HR["Limbs"] == {"T"})].reset_index(drop=True)
# Whole SPT
ACC_response_mean_within_singleLimbs = bursts_ACC_single_limbs.groupby("sub_ID")["HR_response_normalized"].mean()

# t = np.arange(-19, 50)

# for i in range(0, len(subjects)):
#     plt.figure(figsize=(19, 11))
#     plt.plot(t, HR_response_mean_within_singleLimbs[subjects[i]], label = subjects[i])
#     plt.legend()

# All limbs
bursts_ACC_all_limbs = bursts_HR[bursts_HR["Limbs"] == set(("LL", "LW", "RL", "RW", "T"))].reset_index(drop=True)
bursts_ACC_all_limbs_sleep = bursts_ACC_all_limbs[bursts_ACC_all_limbs["SIB"] == 1].reset_index(drop=True)
# divide in low, med and high HR response based on the 1/3 and 2/3 percentiles
ACC_response_percentiles = bursts_ACC_all_limbs.groupby("sub_ID")["AUC"].describe(percentiles=[1/3, 2/3])
AUC_33 = ACC_response_percentiles["33.3%"]
AUC_66 = ACC_response_percentiles["66.7%"]

bursts_ACC_all_limbs["ACC_response_category"] = "medium"
for i, sub in enumerate(subjects):
    bursts_ACC_all_limbs.loc[(bursts_ACC_all_limbs["sub_ID"] == sub) & (bursts_ACC_all_limbs["AUC"] < AUC_33.loc[sub]), "ACC_response_category"] = "low"
    bursts_ACC_all_limbs.loc[(bursts_ACC_all_limbs["sub_ID"] == sub) & (bursts_ACC_all_limbs["AUC"] > AUC_66.loc[sub]), "ACC_response_category"] = "high"

ACC_response_category_mean_within = bursts_ACC_all_limbs.groupby(["sub_ID", "ACC_response_category"])["ACC_response"].mean().unstack()

for i in range(0, len(subjects)):
    plt.figure(figsize=(19, 11))
    plt.plot(t, ACC_response_category_mean_within.loc[subjects[i]]["high"], label = "high")
    plt.plot(t, ACC_response_category_mean_within.loc[subjects[i]]["medium"], label = "medium")
    plt.plot(t, ACC_response_category_mean_within.loc[subjects[i]]["low"], label = "low")
    plt.title(subjects[i])

## All limbs - Sleep vs Wake

In [60]:
bursts_ACC_all_limbs = bursts_HR[bursts_HR["Limbs"] == set(("LL", "LW", "RL", "RW", "T"))].reset_index(drop=True) # all limbs together
# bursts_ACC_all_limbs = bursts_HR # all limbs
bursts_ACC_all_limbs_sleep = bursts_ACC_all_limbs[bursts_ACC_all_limbs["SIB"] == 1].reset_index(drop=True)
bursts_ACC_all_limbs_wake = bursts_ACC_all_limbs[bursts_ACC_all_limbs["SIB"] == 0].reset_index(drop=True)

ACC_response_mean_within_sleep = bursts_ACC_all_limbs_sleep.groupby("sub_ID")["ACC_response"].mean()
ACC_response_mean_within_wake = bursts_ACC_all_limbs_wake.groupby("sub_ID")["ACC_response"].mean()

sem_sleep = ACC_response_mean_within_sleep.to_numpy().std() / np.sqrt(ACC_response_mean_within_sleep.to_numpy().shape[0])
sem_wake = ACC_response_mean_within_wake.to_numpy().std() / np.sqrt(ACC_response_mean_within_wake.to_numpy().shape[0])

t = np.arange(-19, 50)

plt.figure(figsize=(15, 9))
plt.plot(t, ACC_response_mean_within_sleep.mean(), label = "sleep", color = 'blue')
plt.errorbar(t, ACC_response_mean_within_sleep.mean(), yerr = sem_sleep, fmt = '-o', color = 'blue', ecolor='blue', capsize=3)

plt.plot(t, ACC_response_mean_within_wake.mean(), label = "wake", color = 'orange')
plt.errorbar(t, ACC_response_mean_within_wake.mean(), yerr = sem_wake, fmt = '-o', color = 'orange', ecolor='orange', capsize=3)

plt.legend()

## ALL limbs - HIGH vs MED vs LOW

In [65]:
bursts_ACC_all_limbs = bursts_HR[bursts_HR["Limbs"] == set(("LL", "LW", "RL", "RW", "T"))].reset_index(drop=True)

### Whole SPT

In [66]:
# divide in low, med and high HR response based on the 1/3 and 2/3 percentiles
ACC_response_percentiles = bursts_ACC_all_limbs.groupby("sub_ID")["AUC"].describe(percentiles=[1/3, 2/3])
AUC_33 = ACC_response_percentiles["33.3%"]
AUC_66 = ACC_response_percentiles["66.7%"]

bursts_ACC_all_limbs["ACC_response_category"] = "medium"
for i, sub in enumerate(subjects):
    bursts_ACC_all_limbs.loc[(bursts_ACC_all_limbs["sub_ID"] == sub) & (bursts_ACC_all_limbs["AUC"] < AUC_33.loc[sub]), "ACC_response_category"] = "low"
    bursts_ACC_all_limbs.loc[(bursts_ACC_all_limbs["sub_ID"] == sub) & (bursts_ACC_all_limbs["AUC"] > AUC_66.loc[sub]), "ACC_response_category"] = "high"

In [53]:
# coherent averaging within subjects for each category
ACC_response_category_mean_within = bursts_ACC_all_limbs.groupby(["sub_ID", "ACC_response_category"])["ACC_response"].mean().unstack()

sem_high = ACC_response_category_mean_within["high"].to_numpy().std() / np.sqrt(ACC_response_category_mean_within["high"].to_numpy().shape[0])
sem_medium = ACC_response_category_mean_within["medium"].to_numpy().std() / np.sqrt(ACC_response_category_mean_within["medium"].to_numpy().shape[0])
sem_low = ACC_response_category_mean_within["low"].to_numpy().std() / np.sqrt(ACC_response_category_mean_within["low"].to_numpy().shape[0])

t = np.arange(-19, 50, 1)
plt.figure(figsize=(15, 9))
plt.plot(t, ACC_response_category_mean_within["low"].mean(), label = "low", color = 'blue')
plt.errorbar(t, ACC_response_category_mean_within["low"].mean(), yerr = sem_low, fmt = '-o', color = 'blue', ecolor='blue', capsize=3)

plt.plot(t, ACC_response_category_mean_within["medium"].mean(), label = "medium", color = 'orange')
plt.errorbar(t, ACC_response_category_mean_within["medium"].mean(), yerr = sem_medium, fmt = '-o', color = 'orange', ecolor='orange', capsize=3)

plt.plot(t, ACC_response_category_mean_within["high"].mean(), label = "high", color = 'red')
plt.errorbar(t, ACC_response_category_mean_within["high"].mean(), yerr = sem_high, fmt = '-o', color = 'red', ecolor='red', capsize=3)

plt.legend()

## Sleep

In [67]:
bursts_ACC_all_limbs_sleep = bursts_ACC_all_limbs[bursts_ACC_all_limbs["SIB"] == 1].reset_index(drop=True)

In [68]:
# divide in low, med and high HR response based on the 1/3 and 2/3 percentiles
ACC_response_percentiles = bursts_ACC_all_limbs_sleep.groupby("sub_ID")["AUC"].describe(percentiles=[1/3, 2/3])
AUC_33_sleep = ACC_response_percentiles["33.3%"]
AUC_66_sleep = ACC_response_percentiles["66.7%"]

bursts_ACC_all_limbs_sleep["ACC_response_category"] = "medium"
for i, sub in enumerate(subjects):
    bursts_ACC_all_limbs_sleep.loc[(bursts_ACC_all_limbs_sleep["sub_ID"] == sub) & (bursts_ACC_all_limbs_sleep["AUC"] < AUC_33_sleep.loc[sub]), "ACC_response_category"] = "low"
    bursts_ACC_all_limbs_sleep.loc[(bursts_ACC_all_limbs_sleep["sub_ID"] == sub) & (bursts_ACC_all_limbs_sleep["AUC"] > AUC_66_sleep.loc[sub]), "ACC_response_category"] = "high"

In [69]:
# coherent averaging within subjects for each category
ACC_response_category_mean_within_sleep = bursts_ACC_all_limbs_sleep.groupby(["sub_ID", "ACC_response_category"])["ACC_response"].mean().unstack()

sem_high = ACC_response_category_mean_within_sleep["high"].to_numpy().std() / np.sqrt(ACC_response_category_mean_within_sleep["high"].to_numpy().shape[0])
sem_medium = ACC_response_category_mean_within_sleep["medium"].to_numpy().std() / np.sqrt(ACC_response_category_mean_within_sleep["medium"].to_numpy().shape[0])
sem_low = ACC_response_category_mean_within_sleep["low"].to_numpy().std() / np.sqrt(ACC_response_category_mean_within_sleep["low"].to_numpy().shape[0])

t = np.arange(-19, 50, 1)
plt.figure(figsize=(15, 9))
plt.plot(t, ACC_response_category_mean_within_sleep["low"].mean(), label = "low", color = 'blue')
plt.errorbar(t, ACC_response_category_mean_within_sleep["low"].mean(), yerr = sem_low, fmt = '-o', color = 'blue', ecolor='blue', capsize=3)

plt.plot(t, ACC_response_category_mean_within_sleep["medium"].mean(), label = "medium", color = 'orange')
plt.errorbar(t, ACC_response_category_mean_within_sleep["medium"].mean(), yerr = sem_medium, fmt = '-o', color = 'orange', ecolor='orange', capsize=3)

plt.plot(t, ACC_response_category_mean_within_sleep["high"].mean(), label = "high", color = 'red')
plt.errorbar(t, ACC_response_category_mean_within_sleep["high"].mean(), yerr = sem_high, fmt = '-o', color = 'red', ecolor='red', capsize=3)

plt.legend()

## Wake

In [50]:
bursts_ACC_all_limbs_wake = bursts_ACC_all_limbs[bursts_ACC_all_limbs["SIB"] == 0].reset_index(drop=True)

In [51]:
# divide in low, med and high HR response based on the 1/3 and 2/3 percentiles
ACC_response_percentiles = bursts_ACC_all_limbs_wake.groupby("sub_ID")["AUC"].describe(percentiles=[1/3, 2/3])
AUC_33_wake = ACC_response_percentiles["33.3%"]
AUC_66_wake = ACC_response_percentiles["66.7%"]

bursts_ACC_all_limbs_wake["ACC_response_category"] = "medium"
for i, sub in enumerate(subjects):
    bursts_ACC_all_limbs_wake.loc[(bursts_ACC_all_limbs_wake["sub_ID"] == sub) & (bursts_ACC_all_limbs_wake["AUC"] < AUC_33_wake.loc[sub]), "ACC_response_category"] = "low"
    bursts_ACC_all_limbs_wake.loc[(bursts_ACC_all_limbs_wake["sub_ID"] == sub) & (bursts_ACC_all_limbs_wake["AUC"] > AUC_66_wake.loc[sub]), "ACC_response_category"] = "high"

In [52]:
# coherent averaging within subjects for each category
ACC_response_category_mean_within_wake = bursts_ACC_all_limbs_wake.groupby(["sub_ID", "ACC_response_category"])["ACC_response"].mean().unstack()

sem_high = ACC_response_category_mean_within_wake["high"].dropna().to_numpy().std() / np.sqrt(ACC_response_category_mean_within_wake["high"].dropna().to_numpy().shape[0])
sem_medium = ACC_response_category_mean_within_wake["medium"].dropna().to_numpy().std() / np.sqrt(ACC_response_category_mean_within_wake["medium"].dropna().to_numpy().shape[0])
sem_low = ACC_response_category_mean_within_wake["low"].dropna().to_numpy().std() / np.sqrt(ACC_response_category_mean_within_wake["low"].dropna().to_numpy().shape[0])

t = np.arange(-19, 50, 1)
plt.figure(figsize=(15, 9))
plt.plot(t, ACC_response_category_mean_within_wake["low"].dropna().mean(), label = "low", color = 'blue')
plt.errorbar(t, ACC_response_category_mean_within_wake["low"].dropna().mean(), yerr = sem_low, fmt = '-o', color = 'blue', ecolor='blue', capsize=3)

plt.plot(t, ACC_response_category_mean_within_wake["medium"].dropna().mean(), label = "medium", color = 'orange')
plt.errorbar(t, ACC_response_category_mean_within_wake["medium"].dropna().mean(), yerr = sem_medium, fmt = '-o', color = 'orange', ecolor='orange', capsize=3)

plt.plot(t, ACC_response_category_mean_within_wake["high"].dropna().mean(), label = "high", color = 'red')
plt.errorbar(t, ACC_response_category_mean_within_wake["high"].dropna().mean(), yerr = sem_high, fmt = '-o', color = 'red', ecolor='red', capsize=3)

plt.legend()

# Regression

In [28]:
bursts_HR

,Start,End,AUC,PC,transition,Limbs,SIB,sub_ID,ACC_response,HR_response,HR_response_normalized,HR_baseline,HR_peak,HR_peak_latency
0,2024-02-28 23:02:28.765630007,2024-02-28 23:02:31.105629921,6.965556e+03,0.0,None,"{LW, T}",1,158,None,"[81.3763655605152, 79.31300702523212, 77.34783...","[2.9331696699604493, 0.32322226398649434, -2.1...",79.057476,2.933170,6.0
1,2024-02-28 23:03:30.215630054,2024-02-28 23:03:32.105629921,6.534732e+03,0.0,None,"{LW, T}",1,158,"[21.685460424790747, 28.13815298028861, 14.266...","[69.75336400943675, 75.84627452807841, 75.8488...","[-8.115480410659373, -0.08942799216255537, -0....",75.914163,3.951068,44.0
2,2024-02-28 23:08:15.275629997,2024-02-28 23:08:15.635629892,7.761742e+02,0.0,None,{LW},1,158,"[-20.92211175552717, -1.6028287851219574, 1.47...","[73.69902935823436, 78.12130225835978, 78.1519...","[1.685935815921141, 7.787548852055352, 7.82978...",72.477112,9.990368,28.0
3,2024-02-28 23:17:41.875629902,2024-02-28 23:17:42.445630074,1.330175e+03,0.0,None,{LW},1,158,"[-11.010669724082462, 13.495555903064954, 0.63...","[75.84328019638515, 78.12715965978558, 73.6972...","[1.36255449131464, 4.414899484319747, -1.50556...",74.823766,8.753980,7.0
4,2024-02-28 23:31:54.005630016,2024-02-28 23:32:11.155630112,1.179498e+06,0.0,None,"{RW, T, LL, RL, LW}",1,158,"[43.69190609430005, 14.230339833499826, 28.264...","[73.69810238588468, 72.33059595781911, 78.1514...","[-1.7699620455902618, -3.5926712332491917, 4.1...",75.026035,38.834153,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,2024-03-21 06:13:49.784266233,2024-03-21 06:13:56.332916498,1.883960e+05,0.0,None,"{RW, T, LL, RL, LW}",1,815,"[-17.099892658735612, -74.16368991996812, -23....","[52.76078177127652, 52.40690521209172, 51.9092...","[2.313332726315778, 1.6270978198491548, 0.6619...",51.567846,23.109392,6.0
124,2024-03-21 06:15:25.049737692,2024-03-21 06:15:32.442916155,2.045755e+05,0.0,None,"{RW, T, LL, RL, LW}",1,815,"[-26.520409609075372, 14.781811106203321, 48.1...","[43.84215753223191, 48.200524051521654, 48.803...","[-14.243553771483093, -5.718470949524331, -4.5...",51.124037,32.816904,12.0
125,2024-03-21 06:19:12.092283487,2024-03-21 06:19:22.612916231,1.041819e+06,0.0,None,"{RW, T, LL, RL, LW}",0,815,"[-2.8377879804390034, -17.07592119054293, -5.8...","[52.75976035869659, 52.40426872036993, 52.4045...","[-1.514259614286999, -2.1778497626027615, -2.1...",53.570964,30.140481,9.0
126,2024-03-21 06:20:17.472915888,2024-03-21 06:20:23.864266157,6.140845e+04,0.0,None,"{LL, RW, T, RL}",0,815,"[7.613976770865989, 10.286245303647604, 22.261...","[54.988910716309306, 54.988639786929134, 55.37...","[2.9338769839278456, 2.933369830555904, 3.6633...",53.421587,21.802132,10.0


In [92]:
bursts_HR_pairplot = bursts_HR[["HR_peak", "HR_peak_latency", "AUC", "sub_ID", "SIB", "Limbs"]].reset_index(drop=True)

In [93]:
# bursts_HR_pairplot = bursts_HR_pairplot[bursts_HR_pairplot["SIB"] == 1].reset_index(drop=True)
bursts_HR_pairplot = bursts_HR_pairplot[bursts_HR_pairplot["Limbs"] == set(("LL", "LW", "RL", "RW", "T"))].reset_index(drop=True)
bursts_HR_pairplot = bursts_HR_pairplot[bursts_HR_pairplot["sub_ID"] == "971"].reset_index(drop=True)

In [90]:
# drop outliers with AUC too high or too low based on the 5 and 95 percentiles
AUC_percentiles = bursts_HR_pairplot["AUC"].describe(percentiles=[0.05, 0.95])
AUC_5 = AUC_percentiles["5%"]
AUC_95 = AUC_percentiles["95%"]

to_remove = bursts_HR_pairplot[(bursts_HR_pairplot["AUC"] < AUC_5) | (bursts_HR_pairplot["AUC"] > AUC_95)].index
bursts_HR_pairplot = bursts_HR_pairplot.drop(to_remove).reset_index(drop=True)

In [94]:
import seaborn as sns

sns.lmplot(bursts_HR_pairplot, x = "AUC", y = "HR_peak_latency", hue = "sub_ID", col = "SIB")